In [51]:
import pandas as pd
import re

from datasets import Dataset

In [4]:
#Load data - Transcript dataset
transcript_df = pd.read_csv('../data/naruto_transcript.csv')
transcript_df.head()


,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [6]:
#remove actions from transcript
def filter_transcript(text):
    result = re.sub(r'\(.*?\)','',text) #replacing anything inside bracket in text string
    return result

In [10]:
transcript_df['line'] = transcript_df['line'].apply(filter_transcript)
transcript_df.head()

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [17]:
#select naruto lines with atleast n number of words. exclude 1st line here - as we want naruto responding

transcript_df['num_words'] = transcript_df['line'].apply(lambda x: len(x.split()))
transcript_df.head()

,name,line,num_words
0,Naruto,"Give it up. You’re just bent, because you d...",25
1,Hiruzen,I hope you’re not bothering me with some tri...,16
2,Ninja,Naseer Sabah,2
3,Ninja,is the best person on earth,6
4,Naruto,muah,1


In [29]:
transcript_df['naruto_flag']=0
transcript_df.loc[(transcript_df['name']=='Naruto') & (transcript_df['num_words']>5),'naruto_flag']= 1
transcript_df.head()

,name,line,num_words,naruto_flag
0,Naruto,"Give it up. You’re just bent, because you d...",25,1
1,Hiruzen,I hope you’re not bothering me with some tri...,16,0
2,Ninja,Naseer Sabah,2,0
3,Ninja,is the best person on earth,6,0
4,Naruto,muah,1,0


In [ ]:
select_index = list(transcript_df.iloc[1:][transcript_df['naruto_flag']==1].index)  #take only naruto replies. So ignore starting index 0
select_index[:3]

C:\Users\Aayush Bhadani\AppData\Local\Temp\ipykernel_31824\1476494807.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  select_index = list(transcript_df.iloc[1:][transcript_df['naruto_flag']==1].index)


[6, 28, 30]

In [49]:
#system promt for Llama to make it act like our character -> machine will learn to imitiate the response based on examples we provide

system_promt = """ You are Naruto Uzumaki from anime "Naruto". Your response should reflect his personality and speech patterns \n """

prompts = []
for i in select_index:
    prompt = system_promt
    prompt += transcript_df.iloc[i-1]['line']
    prompt += '\n'
    prompt += transcript_df.iloc[i]['line']
    prompts.append(prompt)

print(prompts[7])

 You are Naruto Uzumaki from anime "Naruto". Your response should reflect his personality and speech patterns 
  We will now start the final exam. When your name is called, proceed to the testing room. The final test will be on... The Clone Jutsu.
  Ugh! That’s my worst technique! Great. I’m never gonna pass!


In [50]:
df = pd.DataFrame({'prompt':prompts})
df.head()

,prompt
0,"You are Naruto Uzumaki from anime ""Naruto"". Y..."
1,"You are Naruto Uzumaki from anime ""Naruto"". Y..."
2,"You are Naruto Uzumaki from anime ""Naruto"". Y..."
3,"You are Naruto Uzumaki from anime ""Naruto"". Y..."
4,"You are Naruto Uzumaki from anime ""Naruto"". Y..."


In [52]:
data = Dataset.from_pandas(df)